In [1]:
import json
from pathlib import Path
from typing import List, Dict, Union

def load_file(file_path: Union[str, Path]) -> List[str]:

    path = Path(file_path)

    # JSON 파일 읽기
    with path.open('r', encoding='utf-8') as f:
        predictions = json.load(f)
        
    # dictionary 값들을 리스트로 변환
    if not isinstance(predictions, dict):
        raise ValueError("JSON 파일은 dictionary 형태여야 합니다.")
        
    return list(predictions.values())

In [2]:
ensemble1 = './ensemble_predictions/sudong_clean_bm25_nbest_predictions.json'
ensemble2 = './ensemble_predictions/sudong_clean_final_bm25_nbest_predictions.json'
ensemble3 = './ensemble_predictions/sudong_clean_final_bm25_dpr_nbest_predictions.json'

In [4]:
ensemble4 = './ensemble_predictions/bert_cnn_bm25.json'
ensemble5 = './ensemble_predictions/bert_cnn_dpr.json'
ensemble6 = './ensemble_predictions/bert_cnn_origin.json'

In [5]:
ensemble7 = './ensemble_predictions/tina_cnn_bm25.json'
ensemble8 = './ensemble_predictions/tina_cnn_dpr.json'
ensemble9 = './ensemble_predictions/tina_cnn_origin.json'

In [9]:
ensemble10 = './ensemble_predictions/sudong_origin_bm25.json'
ensemble11 = './ensemble_predictions/sudong_origin_dpr.json'
ensemble12 = './ensemble_predictions/sudong_origin.json'

In [10]:
import json
from collections import defaultdict
from typing import Dict, List, Any

def ensemble_predictions(prediction_files: List[str]) -> Dict[str, str]:
    """
    여러 모델의 nbest_predictions.json 파일을 앙상블하여 최적의 답변을 도출합니다.
    
    Args:
        prediction_files (List[str]): nbest_predictions.json 파일들의 경로 리스트
    
    Returns:
        Dict[str, str]: 각 질문 ID에 대한 최적의 답변
    """
    # 모든 예측을 저장할 딕셔너리
    all_predictions = defaultdict(lambda: defaultdict(float))
    
    # 각 파일의 예측을 처리
    for file_path in prediction_files:
        with open(file_path, 'r', encoding='utf-8') as f:
            predictions = json.load(f)
            
        # 각 질문 ID에 대한 예측을 처리
        for question_id, answers in predictions.items():
            for answer in answers:
                text = answer['text']
                prob = answer['probability']
                # 같은 텍스트에 대한 확률을 더함
                all_predictions[question_id][text] += prob
    
    # 최종 결과를 저장할 딕셔너리
    final_predictions = {}
    
    # 각 질문에 대해 가장 높은 확률을 가진 답변 선택
    for question_id, text_probs in all_predictions.items():
        # 가장 높은 확률을 가진 텍스트 선택
        best_text = max(text_probs.items(), key=lambda x: x[1])[0]
        final_predictions[question_id] = best_text
    
    return final_predictions

def save_predictions(predictions: Dict[str, str], output_file: str):
    """
    최종 예측 결과를 JSON 파일로 저장합니다.
    
    Args:
        predictions (Dict[str, str]): 최종 예측 결과
        output_file (str): 저장할 파일 경로
    """
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(predictions, f, ensure_ascii=False, indent=4)

# 사용 예시
if __name__ == "__main__":
    # 예측 파일 리스트
    prediction_files = [
        ensemble1,
        ensemble2,
        ensemble3,
        ensemble4,
        ensemble5,
        ensemble6,
        ensemble7,
        ensemble8,
        ensemble9,
        ensemble10,
        ensemble11,
        ensemble12,
    ]
    
    # 앙상블 수행
    final_predictions = ensemble_predictions(prediction_files)
    
    # 결과 저장
    save_predictions(final_predictions, "ensemble_predictions_second.json")